In [1]:
import duckdb

In [2]:
# Conectando ao Banco de Dados 'dados_duckdb.db'

con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [3]:
# Visualizando a tabela bronze_z0019 (que está dentro do bd 'dados_duckdb.db')

df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-11-19 22:14:28.898572
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-11-19 22:14:28.898572
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-11-19 22:14:28.898572
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-11-19 22:15:03.155236
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-11-19 22:15:03.155236
5,10003,PREGO,BT10,100,60,z0019_2.csv,2025-11-19 22:15:03.155236


❗ Observa-se que o registro 10003 se repete pois houve alteração de valor na coluna LABST.

✅ Devemos então considerar o último registro.

✅ Precisamos acrescentar uma coluna que adiciona números de forma incremental com base no id (NATBR), pela ordem de data_ingestão (de forma decrescente) .

In [5]:
df2 = con.execute(""" 
                SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row             
                FROM bronze_z0019
                WHERE data_ingestao >= '2025-11-19'
                """).fetchdf()
df2.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10004,SERRA,BT50,100,200,z0019_2.csv,2025-11-19 22:15:03.155236,1
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-11-19 22:14:28.898572,1
2,10003,PREGO,BT10,100,60,z0019_2.csv,2025-11-19 22:15:03.155236,1
3,10003,PREGO,BT10,100,50,z0019_1.csv,2025-11-19 22:14:28.898572,2
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-11-19 22:15:03.155236,1
5,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-11-19 22:14:28.898572,1


❗ Observe que o último registro de cada id (NATBR) recebe o vaor 1 na nova coluna (row)

✅ Vamos considerar somente row = 1

In [6]:
# Filtrando WHERE row=1

df3 = con.execute("""
                  SELECT *
                  FROM( 
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row             
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-11-14'
                  ) WHERE row = 1
                """).fetchdf()
df3.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-11-19 22:14:28.898572,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-11-19 22:15:03.155236,1
2,10003,PREGO,BT10,100,60,z0019_2.csv,2025-11-19 22:15:03.155236,1
3,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-11-19 22:15:03.155236,1
4,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-11-19 22:14:28.898572,1


In [7]:
# Deletando as três última colunas
# Renomeando as demais colunas

df4 = df3.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df4 = df4.rename(columns={"NATBR":"id"})
df4 = df4.rename(columns={"MAKTX":"nm_produto"})
df4 = df4.rename(columns={"WERKS":"id_categoria"})
df4 = df4.rename(columns={"MAINS":"id_fornecedor"})
df4 = df4.rename(columns={"LABST":"vl_preco"})
df4.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT50,100,200
2,10003,PREGO,BT10,100,60
3,10005,MACHADO,BT50,100,100
4,10002,MARTELO,BT50,100,1500


In [8]:
# Verificando o tipo das colunas

df4.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [9]:
# Alterando os tipos das colunas

df_final = df4
df_final = df_final.astype(
    {
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
    }
)

df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10005,MACHADO,BT50,100,100.0
4,10002,MARTELO,BT50,100,1500.0


In [10]:
df_final.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

✅ Para armazenar os dados tratados de df_final, precisamos criar uma tabela

In [11]:
# Criando a tabela produto

con.execute("""
            CREATE TABLE IF NOT EXISTS produtos(
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [12]:
# Verificando a tabela criada (sem registros ainda)

df_resultado2 = con.execute("select * from produtos").fetchdf()
df_resultado2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco


In [13]:
# Inserindo os dados de df_final na tabela produtos

con.execute("INSERT INTO produtos SELECT * FROM df_final")

In [14]:
df_resultado3 = con.execute("select * from produtos").fetchdf()
df_resultado3.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10005,MACHADO,BT50,100,100.0
4,10002,MARTELO,BT50,100,1500.0


In [15]:
con.close()